In [1]:
from pydantic import BaseModel
from typing import List, Optional, Union, Set
from app import create_app, db
from app.models import Akteur
from app.models import Doc as Document
from spacy import displacy 
from pathlib import Path
import seaborn as sns
from logzero import setup_logger
import os

def get_viz(id):
    doc = Document.query.filter_by(id = id).first()
    position_in_res = res.labels.index(id)
    viz = res.viz[position_in_res]
    return doc, viz

def visualize(id):
    doc, viz = get_viz(id)
    ex, options = res.render(doc.text, f"{doc.autor.name} | {doc.autor.party} | {doc.id}", viz, pres=True, online=True)
    displacy.render(ex, style="ent", manual=True, jupyter=True, options=options)

os.environ["FLASK_CONFIG"] = 'development'

app = create_app('default')
app.app_context().push()

In [71]:
import random
from sqlalchemy import and_
doc_rnd = Document.query.filter(and_(Document.autor.has(Akteur.party == 'FDP'), Document.corpus.like('presse'))).all()
sample = random.sample(doc_rnd, 100)

In [18]:
# doc_test = Document.query.filter_by(id=55853).first()

In [72]:
from app.src.d01_ana.analysis import Config, Analysis

settings_analysis = {
    'debug': False,
    'sample': sample,
    'clf_model': 'joeddav/xlm-roberta-large-xnli',
#     'corpus': ['plenar'],
    'pipeline': ['extensions', 'sentiment', 'entity', 'res', 'spans', 'clf']
    }

content_analysis = Analysis('test', Config(**settings_analysis))
content_analysis(to_disk=False, to_db=False)
res = content_analysis.get_results()
res.prepare(post=True)
res.df.sort_values(by='score', ascending=False)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 100/100 [00:11<00:00,  8.93it/s]


,doc,doclen,score,ELITE,PEOPLE,ATTR,ELITE_STANDALONE,E,EA,V,VA
22,54556,157,51.186583,11.878975,8.406693,30.900915,0.000000,11.878975,30.900915,8.406693,0.0
66,54082,182,49.746662,8.731561,13.858664,27.156437,0.000000,8.731561,27.156437,13.858664,0.0
8,54120,526,46.630054,0.000000,13.839395,0.000000,32.790658,32.790658,0.000000,13.839395,0.0
11,54149,627,44.585128,9.560869,18.478038,16.546221,0.000000,9.560869,16.546221,18.478038,0.0
0,54937,114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
33,54355,100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
32,55010,115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
31,54994,163,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
30,54381,423,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [64]:
# res.spans_dict

In [75]:
top = res.df.sort_values(by='score', ascending=False).doc[0:30]
top = [i for i in top]

In [76]:
for i in top:
    visualize(i)

(0, 1050)


(0, 1193)


(0, 3385)


(0, 3848)


(0, 792)


...

(0, 1104)


(0, 1170)


(0, 848)


(0, 3214)


(0, 1128)


In [ ]:
sample_str = '''54491
32861
33161
29713
39051
32461
34235
32130
34634
33420
54908
54475
54810
54103
54021
53813
54015
55031
54852
55018
62073
84771
71576
84267
81681
75286
64906
60307
84265
61206'''
s = sample_str.split('\n')
s = [int(i) for i in s]

# ids = [int(i.split("\n")[0]) for i in sample_str]
sample = [Document.query.filter_by(id=id).one() for id in s]

In [19]:
doc_test = Document.query.filter_by(id=21366).first()

In [20]:
doc_test.corpus

'plenar'

In [18]:
visualize(78180)

(0, 13207)


In [44]:
ex, options = res.render(text, f"{doc.autor.name}", viz, pres=True, online=True)

(0, 3341)


In [46]:
from spacy import displacy 
displacy.render(ex, style="ent", manual=True, jupyter=True, options=options)

In [42]:
res.visualize_jupyter(text, viz, pres=True, online=True)

(0, 3341)


In [ ]:
content_analysis.res_to_disk()